In [1]:
# Document loading, retrieval methods and text splitting
%pip install -qU langchain langchain_community

# Local vector store via Chroma
%pip install -qU langchain_chroma

# Local inference and embeddings via Ollama
%pip install -qU langchain_ollama

# Web Loader
%pip install -qU beautifulsoup4


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install pypdf

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
# Path to the folder containing PDF files
pdf_folder_path = "..\DB\sample-docs"

pdf_loader = PyPDFDirectoryLoader(pdf_folder_path)
print(pdf_loader)

documents = pdf_loader.load()
print(len(documents))

6


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Split text into chunks (chunklarla oynanacak)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(documents)

print(len(documents))

documents[0].page_content
documents[0].metadata

6


{'source': '..\\DB\\sample-docs\\eaquals-accreditation-certificate_sabanci-university-sl_2020.pdf',
 'page': 0}

In [5]:

import onnxruntime
print(onnxruntime.__version__)


1.15.1


c:\Users\idilk\Desktop\su-document-management\myenv-forLLAMA\Lib\site-packages\onnxruntime\capi\_pybind_state.py:26: UserWarning: Please install the 2019 Visual C++ runtime and then try again. If you've installed the runtime in a non-standard location (other than %SystemRoot%\System32), make sure it can be found by setting the correct path.
  warnings.warn("Please install the 2019 Visual C++ runtime and then try again. "


In [8]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings

local_embeddings = OllamaEmbeddings(model="nomic-embed-text")

vectorstore = Chroma.from_documents(documents=documents, embedding=local_embeddings)

In [9]:
from langchain_ollama import ChatOllama

model = ChatOllama(
    model="llama3.2",
)

In [11]:
response_message = model.invoke(
    "what is sabanci universyity about"
)

print(response_message.content)

Sabancı University is a private research university located in Gaziantep, Turkey. It was founded in 1992 by Ömer Sabancı, a Turkish entrepreneur and philanthropist. The university has since grown into one of the top universities in Turkey and the Middle East.

Here are some key facts about Sabancı University:

1. **Academic programs**: Sabancı University offers a wide range of undergraduate and graduate degree programs in various fields, including engineering, natural sciences, social sciences, humanities, and health sciences.
2. **Research focus**: The university has a strong research focus, with a emphasis on interdisciplinary research that addresses real-world problems. It is also a member of the Association of European Universities (EUR).
3. **Facilities and infrastructure**: Sabancı University has invested heavily in modern facilities and infrastructure, including state-of-the-art laboratories, libraries, and research centers.
4. **International collaborations**: The university ha

In [13]:
# template 1 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    "Summarize the main themes in these retrieved docs: {docs}"
)


# Convert loaded documents into strings by concatenating their content
# and ignoring metadata
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = {"docs": format_docs} | prompt | model | StrOutputParser()

question = "tell me about accrediation of Sabanci university"

docs = vectorstore.similarity_search(question)

chain.invoke(docs)

"The retrieved documents appear to be from a university's accreditation report by Eaquals (European Association for Quality Assurance in Higher Education). The main themes that can be extracted from these documents are:\n\n1. **Quality of Teaching**: Sabancı University aims to produce individuals who can think critically, are multidisciplinary, and have strong self-confidence and communication skills.\n2. **Infrastructure and Resources**: The university provides the best physical and digital infrastructure for teaching activities, innovative teaching models, and freedom of program and course selections.\n3. **Student Development and Internationalization**: Sabancı University offers a range of extracurricular social and academic development opportunities, allows students to work closely with academics, and provides international education and development opportunities.\n4. **Social Impact and Responsibility**: All teaching and research activities at Sabancı University aim to contribute 